In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Tiergarten

In [2]:
numNodes = Compute_Jacobian_Tiergarten.numNodes;
numLinks = Compute_Jacobian_Tiergarten.numLinks;
numODpairs = Compute_Jacobian_Tiergarten.numODpairs;
numZones = Compute_Jacobian_Tiergarten.numZones
od_pairs = Compute_Jacobian_Tiergarten.od_pairs;
link_list_js = Compute_Jacobian_Tiergarten.link_list_js;
link_length_list = Compute_Jacobian_Tiergarten.link_length_list;

In [3]:
include("../Julia_files/initia_data.jl");
include("../Julia_files/load_network_uni_class.jl");
include("prepare_data_Tiergarten.jl");
include("extract_data_Tiergarten.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

In [4]:
demandsDict

Dict{Any,Any} with 2 entries:
  0 => Dict{Any,Any}(Pair{Any,Any}((8,23),0.86),Pair{Any,Any}((21,13),9.22),Pai…
  1 => Dict{Any,Any}(Pair{Any,Any}((8,23),0.896638),Pair{Any,Any}((21,13),9.935…

In [5]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [6]:
fcoeffs = [1, 0, 0, 0, 1];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [7]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./benchmark_data/Tiergarten_net.txt", tapFlowDicDict[0]);

In [8]:
arcsDict[0]

Dict{Any,Any} with 766 entries:
  (248,65)  => Arc(248,65,900.0,17.0,494.57)
  (137,106) => Arc(137,106,2800.0,1.66667,223.97)
  (267,256) => Arc(267,256,2400.0,7.33333,86.9503)
  (354,353) => Arc(354,353,900.0,1.0,416.929)
  (56,55)   => Arc(56,55,600.0,0.333333,0.0)
  (259,255) => Arc(259,255,2400.0,0.666667,0.0)
  (125,177) => Arc(125,177,900.0,1.66667,0.0)
  (36,35)   => Arc(36,35,2800.0,0.333333,19.6513)
  (183,184) => Arc(183,184,900.0,0.333333,0.0)
  (43,361)  => Arc(43,361,2800.0,12.6667,6.53038)
  (16,277)  => Arc(16,277,999999.0,0.0,1556.27)
  (298,307) => Arc(298,307,2400.0,1.0,1529.74)
  (31,34)   => Arc(31,34,2800.0,0.666667,106.18)
  (187,191) => Arc(187,191,600.0,0.666667,228.489)
  (241,244) => Arc(241,244,2400.0,0.333333,0.0)
  (84,95)   => Arc(84,95,900.0,2.33333,0.0)
  (221,2)   => Arc(221,2,999999.0,0.0,739.205)
  (23,351)  => Arc(23,351,999999.0,0.0,0.0)
  (159,200) => Arc(159,200,2400.0,12.6667,0.0)
  (96,92)   => Arc(96,92,900.0,2.0,0.0)
  (35,39)   => Arc(35,39,

In [9]:
deg = 6
c = .5
lam = .001

coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDict[1], arcsDict[0]);

In [10]:
# weight parameters
gamma1 = 1
gamma2 = 1

1

In [11]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], coeffs_dict[(deg, c, lam, 1)]);

demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

In [12]:
demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [13]:
# get arcs data corresponding to initial demands and flows
arcsDict[1] = observFlow("./benchmark_data/Tiergarten_net.txt", tapFlowDicDict[1]);

In [14]:
linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [15]:
linkCostDicDict[1]["0"], link_length_list[1]

(0.0,0.0)

In [16]:
jacobiSpiessDict[1] = Compute_Jacobian_Tiergarten.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

In [17]:
# maximum number of iterations
N = 100;  

# Armijo rule parameters
rho = 2;
M = 10;

# search direction parameter
epsilon_1 = 0;

# stop criterion parameter
epsilon_2 = 1e-20;

In [18]:
for l = 1:N
    
    jacobDict[l] = jacobiSpiessDict[l]
    
    demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

    descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[1], tapFlowVecDict[l], 
        tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

    searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

    thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

    demandsVecDict[l+1] = similar(demandsVecDict[0]);
    
    demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
        demandsVecDict[1], coeffs_dict[(deg, c, lam, l)], searchDirecDict[l], thetaMaxDict[l], rho, M);
            
    demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);
    
    tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], coeffs_dict[(deg, c, lam, l)]);
        
    arcsDict[l+1] = observFlow("./benchmark_data/Tiergarten_net.txt", tapFlowDicDict[l+1]);
        
    coeffs_dict[(deg, c, lam, l+1)], objInvVIDict[l+1] = train(lam, deg, c, demandsDict[l+1], arcsDict[0]);
    
#     objFunTmp = objF(gamma1, gamma2, demandsVecDict[l+1], demandsVecDict[1], 
#         coeffs_dict[(deg, c, lam, l+1)])
    
#     if objFunTmp > objFunDict[l+1]
#         coeffs_dict[(deg, c, lam, l+1)] = coeffs_dict[(deg, c, lam, l)]
#     end

    linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], coeffs_dict[(deg, c, lam, l+1)]);
    
    jacobiSpiessDict[l+1] = Compute_Jacobian_Tiergarten.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);
        
    demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);
    
    # stopping criterion
    if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
        break
    end
  
end

# normalize objective function value
for l = 1:(length(objFunDict))
    norObjFunDict[l] = objFunDict[l] / objFunDict[1];
end

In [19]:
demandsVecDict

Dict{Any,Any} with 17 entries:
  2  => [22.9198,16.5206,19.9205,14.6782,3.86915,0.119719,0.3002,1.21731,0.5055…
  16 => [22.9198,16.7841,20.1374,15.0826,4.5304,0.904903,0.629332,1.5954,0.9628…
  11 => [22.9272,16.66,20.0399,14.9079,4.22199,0.541801,0.496286,1.43868,0.7719…
  0  => [22.59,18.32,21.82,15.22,3.73,0.1,0.31,1.22,0.52,0.21  …  7.49,11.8,2.4…
  7  => [22.9263,16.6058,19.9949,14.8238,4.08558,0.379447,0.428582,1.36093,0.67…
  9  => [22.931,16.6413,20.025,14.8801,4.17604,0.487253,0.474278,1.41325,0.7410…
  10 => [22.9301,16.6503,20.0326,14.8937,4.1984,0.513842,0.484968,1.42562,0.756…
  8  => [22.933,16.6248,20.0109,14.8543,4.13395,0.437172,0.45365,1.38951,0.7121…
  6  => [22.9258,16.5973,19.9872,14.8099,4.06364,0.353178,0.417194,1.34792,0.66…
  4  => [22.9248,16.5595,19.9545,14.7457,3.96667,0.236847,0.36149,1.28534,0.586…
  3  => [22.9215,16.5315,19.9301,14.6973,3.89654,0.152646,0.317727,1.23673,0.52…
  5  => [22.9248,16.5595,19.9545,14.7457,3.96667,0.236847,0.36149,1.28534,0.58

In [20]:
norObjFunDict

Dict{Any,Any} with 16 entries:
  2  => 0.987111
  16 => 0.472406
  11 => 0.684384
  7  => 0.798019
  9  => 0.719714
  10 => 0.695136
  8  => 0.758075
  6  => 0.822445
  4  => 0.914395
  3  => 0.968035
  5  => 0.905607
  13 => 0.540745
  14 => 0.473039
  15 => 0.472406
  12 => 0.591785
  1  => 1.0

In [21]:
coeffs_dict

Dict{Any,Any} with 16 entries:
  (6,0.5,0.001,7)  => [1.0,0.0157331,0.0915365,0.218496,0.28066,0.19333,0.05698…
  (6,0.5,0.001,8)  => [1.0,0.0157345,0.091513,0.218395,0.280489,0.19319,0.05693…
  (6,0.5,0.001,1)  => [1.0,0.0153576,0.0892163,0.21304,0.2738,0.188699,0.055645…
  (6,0.5,0.001,2)  => [1.0,0.0153924,0.0894345,0.213558,0.274456,0.189145,0.055…
  (6,0.5,0.001,15) => [1.0,0.0146469,0.0885006,0.221391,0.300662,0.220014,0.069…
  (6,0.5,0.001,6)  => [1.0,0.0157044,0.091362,0.21809,0.280156,0.192995,0.05688…
  (6,0.5,0.001,14) => [1.0,0.0143455,0.0875938,0.220825,0.301843,0.222086,0.070…
  (6,0.5,0.001,12) => [1.0,0.0159571,0.0928183,0.221346,0.284073,0.195531,0.057…
  (6,0.5,0.001,11) => [1.0,0.0159817,0.0931053,0.222211,0.285355,0.196514,0.057…
  (6,0.5,0.001,3)  => [1.0,0.015445,0.0897657,0.214344,0.275453,0.189823,0.0559…
  (6,0.5,0.001,4)  => [1.0,0.0155356,0.0903077,0.2156,0.277015,0.190868,0.05627…
  (6,0.5,0.001,10) => [1.0,0.0159724,0.0930599,0.222123,0.285263,0.196463,0.05

In [22]:
outfile = open("./results/norObjFunDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, norObjFunDict)

close(outfile)

In [23]:
outfile = open("./results/demandsDiffDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDiffDict)

close(outfile)

In [24]:
outfile = open("./results/objInvVIDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, objInvVIDict)

close(outfile)

In [25]:
outfile = open("./results/coeffs_dict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)

In [26]:
outfile = open("./results/demandsVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsVecDict)

close(outfile)

In [27]:
outfile = open("./results/demandsDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDict)

close(outfile)

In [28]:
outfile = open("./results/tapFlowDicDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [29]:
outfile = open("./results/tapFlowVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowVecDict)

close(outfile)